In [65]:

from SimPEG import*
import SimPEG.PF as PF

H0 = (50000., 60., 270.)
b0 = PF.MagAnalytics.IDTtoxyz(-H0[1], H0[2], H0[0])
cs = 0.2
rad = 2.
hxind = [(cs, 21)]
hyind = [(cs, 21)]
hzind = [(cs, 21)]
mesh = Mesh.TensorMesh([hxind, hyind, hzind], 'CCC')

chi = 0.01
ndv = -100

# Get cells inside the sphere
sph_ind = PF.MagAnalytics.spheremodel(mesh, 0., 0., 0., rad)

# Adjust susceptibility for volume difference
Vratio = (4./3.*np.pi*rad**3.) / (np.sum(sph_ind)*cs**3.)
model = np.ones(mesh.nC)*chi*Vratio

model = model[sph_ind]

actvMap = Maps.InjectActiveCells(mesh, sph_ind, ndv)
# Creat reduced identity map
idenMap = Maps.IdentityMap(nP=int(sum(sph_ind)))

# Create plane of observations
xr = np.linspace(-20, 20, 21)
yr = np.linspace(-20, 20, 21)
X, Y = np.meshgrid(xr, yr)

# Move obs plane 2 radius away from sphere
Z = np.ones((xr.size, yr.size))*2.*rad
locXyz = np.c_[Utils.mkvc(X), Utils.mkvc(Y), Utils.mkvc(Z)]
ndata = locXyz.shape[0]
rxLoc = PF.BaseMag.RxObs(locXyz)
srcField = PF.BaseMag.SrcField([rxLoc], param=H0)
survey = PF.BaseMag.LinearSurvey(srcField)


prob = PF.Magnetics.MagneticIntegral(mesh, mapping=idenMap,
                                     actInd=sph_ind, forwardOnly=True,
                                     rtype='xyz')

survey.pair(prob)

    #def test_ana_forward(self):

d = prob.fields(model)

dbx = d[0:ndata]
dby = d[ndata:2*ndata]
dbz = d[2*ndata:]

prob = PF.Magnetics.MagneticIntegral(mesh, mapping=idenMap,
                                     actInd=sph_ind, forwardOnly=True,
                                     rtype='tmi')
survey.pair(prob)
dtmi = prob.fields(model)

bxa, bya, bza = PF.MagAnalytics.MagSphereFreeSpace(locXyz[:, 0], locXyz[:, 1], locXyz[:, 2], rad, 0, 0, 0, chi, b0)

# Project data to tmi
D = (450.-float(H0[2])) % 360.
I = H0[1]

# Projection matrix
Ptmi = mkvc(b0)/np.sqrt(np.sum(b0**2.))

btmi = mkvc(Ptmi.dot(np.vstack((bxa, bya, bza))))


#dpred = survey.projectFieldsAsVector(B)
#err = np.linalg.norm(dpred-np.r_[bxa, bya, bza])/np.linalg.norm(np.r_[bxa, bya, bza])
#self.assertTrue(err < 0.08)

Begin calculation forward calculations... G not stored: 
Done 0.0 %
Done 10.0 %
Done 20.0 %
Done 30.0 %
Done 40.0 %
Done 50.0 %
Done 60.0 %
Done 70.0 %
Done 80.0 %
Done 90.0 %
Done 100% ...forward operator completed!!

Begin calculation forward calculations... G not stored: 
Done 0.0 %
Done 10.0 %
Done 20.0 %
Done 30.0 %
Done 40.0 %
Done 50.0 %
Done 60.0 %
Done 70.0 %
Done 80.0 %
Done 90.0 %
Done 100% ...forward operator completed!!



In [66]:
err_xyz = np.linalg.norm(d-np.r_[bxa, bya, bza])/np.linalg.norm(np.r_[bxa, bya, bza])
err_tmi = np.linalg.norm(dtmi-btmi)/np.linalg.norm(btmi)
print err_xyz, err_tmi,

0.00201220840839 0.00207989757317
